In [ ]:
# import module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 50)
%matplotlib inline

# Extract Data

In [ ]:
### Extract Data Part ###
# Import datasets
# Listing 
df = pd.read_csv('listings_detail.csv')
# NYC ZipCode
df_zipcode = pd.read_csv('nyc_zip.csv')

# Transform Data

In [ ]:
# Select Columns
df = df[['id','neighborhood_overview','host_response_rate',
'host_is_superhost','host_total_listings_count','host_has_profile_pic',
'host_identity_verified','zipcode','property_type',
'room_type','accommodates','bathrooms',
'bedrooms','beds','price',
'minimum_nights','maximum_nights','number_of_reviews',
'review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
'review_scores_checkin','review_scores_communication','review_scores_location',
'review_scores_value']]

# Show first 5rows
# df.head()

# Check data types
# df.dtypes

# Count Null data
#df.isnull().sum()

### Cleaning Data ###
# Drop NULL
df_drop_na = df.dropna()

# Remove Strings
df_drop_na['host_response_rate'] = df_drop_na['host_response_rate'].str.strip('%')
df_drop_na['price'] = df_drop_na['price'].str.strip('$')
df_drop_na['price'] = df_drop_na['price'].str.replace(',','')

# Convert boolean to int
arr_mapping = {'t':1, 'f':0}
df_drop_na['host_is_superhost'] = df_drop_na['host_is_superhost'].map(arr_mapping)
df_drop_na['host_has_profile_pic'] = df_drop_na['host_has_profile_pic'].map(arr_mapping)
df_drop_na['host_identity_verified'] = df_drop_na['host_identity_verified'].map(arr_mapping)

# Zipcode - Align 5 numbers
list_zipcode = df_drop_na['zipcode']
new_zipcode = []
for i in list_zipcode:
    if type(i) is str:
        if len(i)<5:
            new_zipcode.append(None)
        else:
            j = i[:5]
            j = int(j)
            new_zipcode.append(j)
    elif type(i) is float:
        j = int(i)
        new_zipcode.append(j)
    else:
        new_zipcode.append(i)

# Switch Zipcode data
df_drop_na['zipcode'] = new_zipcode

#drop rows have NULL data 
df_drop_na = df_drop_na.dropna()

# Change the datatypes
df_drop_na['zipcode'] = df_drop_na['zipcode'].astype(np.int64)
df_drop_na['price'] = df_drop_na['price'].astype(np.float32)

# New Column
df_drop_na['Price_daily'] = df_drop_na['price']/df_drop_na['minimum_nights']

# Merge two datasets
df_merge = pd.merge(df_drop_na, df_zipcode,  left_on='zipcode', right_on='zip' )

# Export modified datasets
# df_merge.to_csv('df_merge.csv')

# Load Data

In [ ]:
import pymysql 

connection = pymysql.connect(host="127.0.0.1",user="root",password="cis4400", db="airbnb", cursorclass=pymysql.cursors.DictCursor)
print(connection)

sql ="CREATE TABLE if not exists airbnb_details(id INT, host_response_rate INT, host_is_superhost BOOLEAN, host_total_listings_count INT, host_has_profile_pic BOOLEAN, host_identity_verified BOOLEAN, zipcode INT, property_type VARCHAR(255), room_type VARCHAR(255), accomodates INT, bathroom INT, bedrooms INT, bed INT, price INT, minimum_nights INT, maximum_nights INT, number_of_reviews INT, reviews_scores_rating INT,review_scores_accuracy INT,review_scores_cleanliness INT, review_scores_checkin INT, review_scores_communication INT,review_scores_location INT, review_scores_value INT);"
sql2="CREATE TABLE if not exists nyc_zipcodes(zipcode INT, borough VARCHAR(255), neighborhood VARCHAR(255), population INT, density INT); "
sql3="Load data local infile 'modified_airbnb_data.csv' INTO TABLE airbnb.airbnb_details fields terminated by "," IGNORE 1 lines;"
sql4="Load data local infile 'og_nyc_zipcodes.csv' INTO TABLE airbnb.nyc_zipcodes fields terminated by "," IGNORE 1 lines; "
sql5="show tables;"


tables = [sql, sql2]

def create_tables(tables):
for i in tables:
    cursor = connection.cursor()
    cursor.execute(i)
    connection.commit()

connection.close()

data = [sql3, sql4]
def load_data(data):
    for x in data:
        cursor = connection.cursor()
        cursor.execute(x)
        connection.commit()


create_tables(tables)
load_data(data)